In [1]:
import pandas as pd
import numpy as np
from eda.tools import seq_to_num

In [2]:
train_df = pd.read_csv('data/train.csv', index_col=0)
X_train = seq_to_num(train_df.Sequence, pad=False, target_split=False)

Using TensorFlow backend.


In [ ]:
rnn_filter = lambda seq: len(seq) > 2 and np.all([0 <= x < 2000 for x in seq])
X_train = X_train[X_train.map(rnn_filter)]

In [ ]:
def prep_data(data, seqlen, frac=.7):
    """
    Generate train dataset, given sequences
    for each sequence, we iterate creating input sequence and target term
    """
    X, y = [], []
    for seq in data:
        if len(seq) <= seqlen:
            X += [list(map(int, [0] * (seqlen - (len(seq) - 1)) + seq[:-1].tolist()))]
            y += [int(seq[-1])]
            continue
        x1 = [seq[i: i + seqlen] for i in range(len(seq) - seqlen)]
        y1 = list(map(int, seq[seqlen:].tolist()))
        X += x1
        y += y1
    X = np.array(X)
    X = np.expand_dims(X, 2)
    y = np.array(y)
    y = np.expand_dims(y, 1)
    return X, y